<a href="https://colab.research.google.com/github/sujitha-msit/CGL3/blob/main/melspectrum_Turnkey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sujithajallepalli","key":"f4e5848babd68d7955dfaa460110415d"}'}

In [ ]:
!mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c '11785-Spring2021-Hw1P2'

 97% 314M/322M [00:14<00:00, 38.0MB/s]
100% 322M/322M [00:14<00:00, 23.7MB/s]
100% 3.51G/3.52G [01:50<00:00, 80.6MB/s]
100% 3.52G/3.52G [01:50<00:00, 34.3MB/s]
  0% 0.00/3.43M [00:00<?, ?B/s]
100% 3.43M/3.43M [00:00<00:00, 94.1MB/s]
  0% 0.00/500k [00:00<?, ?B/s]
100% 500k/500k [00:00<00:00, 151MB/s]
 96% 297M/310M [00:10<00:00, 25.2MB/s]
100% 310M/310M [00:10<00:00, 30.4MB/s]
 92% 130M/142M [00:01<00:00, 79.7MB/s]
100% 142M/142M [00:01<00:00, 107MB/s] 
  0% 0.00/518 [00:00<?, ?B/s]
100% 518/518 [00:00<00:00, 413kB/s]


In [ ]:
!mkdir test
!mkdir train
!mkdir dev

In [ ]:
! unzip train.npy.zip -d train
!unzip test.npy.zip -d test
!unzip dev.npy.zip -d dev


Archive:  train.npy.zip
  inflating: train/train.npy         
Archive:  test.npy.zip
  inflating: test/test.npy           
Archive:  dev.npy.zip
  inflating: dev/dev.npy             


In [ ]:
!unzip dev_labels.npy.zip -d dev

Archive:  dev_labels.npy.zip
replace dev/dev_labels.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dev/dev_labels.npy      


In [ ]:
import numpy as np
!rm *.npy.zip

rm: cannot remove 'train.npy.zip': No such file or directory


In [ ]:
import numpy as np

train_data=np.load('train/train.npy',allow_pickle=True)
train_labels=np.load('train_labels.npy',allow_pickle=True)
# test_data=np.load('test/test.npy',allow_pickle=True)
# validation_data=np.load('dev/dev.npy',allow_pickle=True)
# validation_labels=np.load('dev/dev_labels.npy',allow_pickle=True)

In [ ]:
# print("train_data",train_data[1])
# print("test_data",test_data[1].shape)
# print("train_labels",train_labels[1].shape)
# print("validation_data",validation_data[1].shape)
# print("validation_labels",validation_labels[1].shape)

In [ ]:
# import numpy as np
# np.vstack(data_x)
# data_x.shape

(2,)

In [ ]:
# validation_data.shape
# validation_labels.shape

(2200,)

In [ ]:
import numpy as np
train_data=np.concatenate(train_data,axis=0)



NameError: ignored

In [ ]:
import numpy as np
import torch
torch.utils.data.Dataset
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, Y, context):
        # Store paramters as class variables
        self.X=X
        self.Y=Y
        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        da=[]
        for x in X:
          for i in x:
            da.append(i)

        self.X=np.array(da,dtype=np.float)
      #  padding the data at starting of the data and ending of  the data with context no of rows
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
        da=[]
        for y in Y:
          for j in y:
            da.append(j)
        self.Y=np.array(da,dtype=np.float)
        # print("self.Y at index 0 istype is:",self.Y[0])




    def __len__(self):
        # print(self.Y.shape[0])
        return len(self.Y)

    def __getitem__(self,index):


        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        # print(xx)
        # xx=self.X[index]
        yy=self.Y[index]
        return xx,yy
    def collate_fn(batch):

        ### Select all data from batch (1 line)
        batch_x = [x for x,y in batch]
        # print(batch_x)
        ### Select all labels from batch (1 line)
        batch_y = [y for x,y in batch]

        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        batch_y = torch.as_tensor(batch_y)

        ### Return batched data and labels (1 line)
        return batch_x, batch_y


In [ ]:
# dataset4 = MyDataset(data_x, data_y,context=2)

# dataloader4 = torch.utils.data.DataLoader(dataset4,
#                                           batch_size=3,
#                                           shuffle=False,
#                                           collate_fn=MyDataset.collate_fn)

# for i, batch in enumerate(dataloader4):
#   if i==0:
#     print("Batch", i, ":\n", batch, "\n")

In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=5
dataset=MyDataset(train_data,train_labels,context)
train_loader_args = dict(shuffle=True, batch_size=512, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)


In [ ]:
import torch.nn as nn
class Simple_MLP(nn.Module):
    def __init__(self,input_output_data):
        super(Simple_MLP, self).__init__()
        self.input_output_data=input_output_data
        layers=[]
        for i in range(len(self.input_output_data)-2):
          layers.append(nn.Linear(input_output_data[i],input_output_data[i+1]))
          layers.append(nn.ReLU())
        layers.append(nn.Linear(input_output_data[-2],input_output_data[-1]))

        self.net=nn.Sequential(*layers)

    def forward(self, x):
        # make sure to return the output after
        # call the network created above
        return self.net(x.float())



In [ ]:
import torch.optim as optim
model=Simple_MLP([440,320,225,71])
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters())
device=torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

In [ ]:
import time
def train_epoch(model, train_loader, criterion, optimizer):

  print("Training....")
  model.train()

  running_loss=0.0
  total_predictions=0.0
  correct_predictions=0.0
  start_time=time.time()

  for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
    optimizer.zero_grad() #hackward() accumulates gradients
    data=data.to(device)

    target=target.to(device) #data & sodel on same device

    outputs=model(data)
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    loss.backward()
    optimizer.step()


    # Here the accuracy can be calculaated with the no training examples being
    # correclty predicted i,e target is the y and outputs is y^)
    # correct_predictions=torch.sum(outputs==target)
    # total_predictions=outputs.shape[0]
    # print(accuracy)
    # print(outputs.shape,target.shape)

    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    # print(value,index,"--------------------")

    # print(outputs[1],target[1],"===============================","index is :",value)
    # if index==target[1]:
    #   print(index,target[1])
    # # ,target[1])


    end_time=time.time()

    running_loss/=len(train_loader)

    print("Training Loss:", running_loss, "Time: ",end_time-start_time,'s')

    acc=(correct_predictions/total_predictions)*100.0


    print('Training Accuracy:', acc, "% ")
  return running_loss ,acc


In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=5
val_dataset=MyDataset(validation_data,validation_labels,context)
val_loader_args = dict(shuffle=False, batch_size=512, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=64)
val_loader=torch.utils.data.DataLoader(val_dataset,**val_loader_args)


In [ ]:
# validation data model lis tested.
import time
def val_epoch(model, val_loader, criterion, optimizer):

  print("validating....")
  model.eval()

  running_loss=0.0
  total_predictions=0.0
  correct_predictions=0.0
  start_time=time.time()

  for batch_idx, (data, target) in enumerate(train_loader):
    print(batch_idx)
    optimizer.zero_grad() #hackward() accumulates gradients
    data=data.to(device)

    target=target.to(device) #data & sodel on same device

    outputs=model(data)
    loss=criterion(outputs, target.long())
    running_loss+=loss.item()

    loss.backward()
    optimizer.step()


    # Here the accuracy can be calculaated with the no training examples being
    # correclty predicted i,e target is the y and outputs is y^)
    # correct_predictions=torch.sum(outputs==target)
    # total_predictions=outputs.shape[0]
    # print(accuracy)
    # print(outputs.shape,target.shape)

    value,index=torch.max(outputs,1)
    correct_predictions=torch.sum(index==target)
    total_predictions=outputs.shape[0]
    # print(value,index,"--------------------")

    # print(outputs[1],target[1],"===============================","index is :",value)
    # if index==target[1]:
    #   print(index,target[1])
    # # ,target[1])


    end_time=time.time()

    running_loss/=len(train_loader)

    print("testing Loss:", running_loss, "Time: ",end_time-start_time,'s')

    acc=(correct_predictions/total_predictions)*100.0


    print('Testing Accuracy:', acc, "% ")
  return running_loss ,acc


In [ ]:
train_epoch(model, train_loader, criterion, optimizer)

In [ ]:
val_epoch(model, val_loader, criterion, optimizer)

In [ ]:
import numpy as np
import torch
torch.utils.data.Dataset
class MyDatasettest(torch.utils.data.Dataset):
    def __init__(self, X, context):
        # Store paramters as class variables
        self.X=X

        self.context=context
        # Taking the data in variable x and arraning all the frames in one list
        da=[]
        for x in X:
          for i in x:
            da.append(i)

        self.X=np.array(da,dtype=np.float)
      #  padding the data at starting of the data and ending of  the data with context no of rows
        self.length=self.X.shape[0]
        self.X=np.pad(self.X,pad_width=((self.context,self.context),(0,0)),mode="constant",constant_values=0)
        # da=[]
        # for y in Y:
        #   for j in y:
        #     da.append(j)
        # self.Y=np.array(da,dtype=np.float)
        # print("self.Y at index 0 istype is:",self.Y[0])




    def __len__(self):
        # print(self.Y.shape[0])
        return self.length

    def __getitem__(self,index):


        start_index=index
        end_index=index+2*self.context+1
        xx=self.X[start_index:end_index,:].flatten()
        # print(xx)
        # xx=self.X[index]
        # yy=self.Y[index]
        return xx
    def collate_fn(batch):

        ### Select all data from batch (1 line)
        batch_x =batch
        # print(batch_x)
        ### Select all labels from batch (1 line)
        # batch_y = [y for x in batch]

        ### Convert batched data and labels to tensors (2 lines)
        batch_x = torch.as_tensor(batch_x)
        # batch_y = torch.as_tensor(batch_y)

        ### Return batched data and labels (1 line)
        return batch_x


In [ ]:
cuda = torch.cuda.is_available()

num_workers=2 if cuda else 0
context=5
dataset=MyDatasettest(test_data,context)
train_loader_args = dict(shuffle=True, batch_size=512, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
test_loader=torch.utils.data.DataLoader(dataset,**train_loader_args)


In [ ]:

# test data model is tested.
import time
def test_epoch(model, test_loader):
  indexes=[]
  print("testing data is taken....")
  model.eval()

  running_loss=0.0
  total_predictions=0.0
  correct_predictions=0.0
  start_time=time.time()
  result=[]

  for batch_idx, data in enumerate(test_loader):

    data=data.to(device)
    outputs=model(data)
    # print(outputs.shape)
    value,index=torch.max(outputs,1)
    result.append(index.tolist())
  return result

    # Here the accuracy can be calculaated with the no training examples being
    # correclty predicted i,e target is the y and outputs is y^)
    # correct_predictions=torch.sum(outputs==target)
    # total_predictions=outputs.shape[0]
    # print(accuracy)
    # print(outputs.shape,target.shape)


    # print(value,index,"--------------------")

    # print(outputs[1],target[1],"===============================","index is :",value)
    # if index==target[1]:
    #   print(index,target[1])
    # # ,target[1])




result=test_epoch(model, test_loader)

In [ ]:
tensor_result=torch.as_tensor(result[0:-1]).flatten()
tensor_result.shape
last=torch.as_tensor(result[-1])
final_tensor=torch.hstack((tensor_result,last))
final=final_tensor.tolist()
from pandas import DataFrame
# your_list = ['item1', 'item2', 'item3',...]
df = DataFrame (final,columns=['labels'])
df.to_csv("sample.csv",index_label='id')
df.head



In [ ]:
import torch
import numpy as np
X = np.array([ np.array([[ 2,  3,  4],
                         [ 4,  6,  8],
                         [ 6,  9, 12],
                         [ 8, 12, 16]]),
               np.array([[10, 15, 20],
                         [12, 18, 24]]) ], dtype=object)

Y = np.array([ np.array([1, 2, 3, 4]),
               np.array([5, 6])], dtype=object)
X.shape
np.vstack(X)

array([[ 2,  3,  4],
       [ 4,  6,  8],
       [ 6,  9, 12],
       [ 8, 12, 16],
       [10, 15, 20],
       [12, 18, 24]])